In [ ]:
!jupyter nbextension enable --py widgetsnbextension
from ipywidgets import FloatProgress
import pandas as pd
df = pd.DataFrame({'A': [1, 2, 3, 4], 'B': [1, 2, 3, 4]})


In [ ]:
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_widgets()

import pandas as pd
from tqdm.notebook import tqdm

tqdm.pandas()

df = pd.DataFrame({"a": ["foo", "bar"], "b": ["spam", "eggs"]})
df.progress_apply(lambda row: row["a"] + row["b"], axis=1)


In [ ]:
!pip install datasets 
!pip install --no-cache-dir --upgrade gdown
# !gdown 1spSFY1yieMcjGJc989bbbEheNhegmEmR
# !7z e torob-data-challenge-2023_datafiles_v1.7z
!pip install parsivar --no-cache-dir
!pip install hazm --no-cache-dir
!pip install transformers 
!pip install python-negar --no-cache-dir --upgrade
!pip install -q sentencepiece

In [ ]:
!pip install datasets  --no-cache-dir
!pip install transformers  --no-cache-dir

In [ ]:
import json
import numpy as np
import pandas as pd

from parsivar import Normalizer
from negar.virastar import PersianEditor
import hazm

from collections import defaultdict, Counter
from tqdm.notebook import tqdm_notebook
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset, DatasetDict
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from sklearn.metrics import accuracy_score
from transformers import AdamW


In [ ]:
class MyNormalizer:
    def __init__(self):
        self.parsivar_normalizer = Normalizer(pinglish_conversion_needed=True)
        self.hazm_normalizer = hazm.Normalizer(
            remove_extra_spaces=True,
            persian_numbers=False,
            persian_style=False,
            punctuation_spacing=False,
            remove_diacritics=True,
            affix_spacing=True,
            token_based=True,
        )

    def normilize(self, txt):
        return self.hazm_normalizer.normalize(
            self.parsivar_normalizer.normalize(txt.replace("\n", " ").lower().strip())
        )


In [ ]:
class JsonFileIterator:
    def __init__(self, path):
        self.path = path
        self.f = open(path, "r")
        self.i = 0
        self.length = self.counter_lines()

    def __iter__(self):
        return self

    def __next__(self):
        line = self.f.readline()
        if not line:
            # End of file
            self.f.close()
            raise StopIteration
        self.i += 1
        return json.loads(line)

    def counter_lines(self):
        with open(self.path, "r") as f1:
            return sum(1 for _ in f1)

    def __len__(self):
        return self.length


In [ ]:
normalizer = MyNormalizer()
search_data = JsonFileIterator("./data/torob-search-data_v1.jsonl")
queries = dict()
for search in tqdm_notebook(search_data):
    raw_query = search["raw_query"]
    normalized_query = normalizer.normilize(raw_query)
    if queries.get(normalized_query, -1) == -1:
        queries[normalized_query] = 1
    else:
        queries[normalized_query] += 1


In [ ]:
len(queries)

In [ ]:
agg_searches = defaultdict(
    lambda: dict(
        results=Counter(),
        clicks=Counter(),
    )
)

search_data = JsonFileIterator("./data/torob-search-data_v1.jsonl")
print("Aggregating searches based on raw query...")

for search in tqdm_notebook(search_data):
    raw_query = search["raw_query"]
    normalized_query = normalizer.normilize(raw_query)

    if queries[normalized_query] > 10:
        results = search["result"][: np.max(search["clicked_rank"]) + 8]
        clicked_results = search["clicked_result"]
        agg_searches[normalized_query]["results"].update(results)
        agg_searches[normalized_query]["clicks"].update(clicked_results)


In [ ]:
len(agg_searches)

In [ ]:
product_info = JsonFileIterator("./data/products-info_v1.jsonl")

In [ ]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect("my_database.db")
c = conn.cursor()

# Create a table to store the data
c.execute(
    "CREATE TABLE products (id INTEGER PRIMARY KEY, category_name TEXT, titles TEXT, min_price REAL, max_price REAL, avg_price REAL, min_num_shops REAL, max_num_shops REAL, avg_num_shops REAL)"
)


# Insert the data into the table
for product in tqdm_notebook(product_info):
    titles_json = json.dumps(product["titles"])
    c.execute(
        "INSERT INTO products (id, category_name, titles, min_price, max_price, avg_price, min_num_shops, max_num_shops, avg_num_shops) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
        (
            product["id"],
            product["category_name"],
            titles_json,
            product["min_price"],
            product["max_price"],
            product["avg_price"],
            product["min_num_shops"],
            product["max_num_shops"],
            product["avg_num_shops"],
        ),
    )
# Commit the changes and close the database connection
conn.commit()
conn.close()


In [ ]:
def find_new_words(titles_list, index_high_len):
    highest_len_title_words = titles_list[index_high_len].split()
    total_new_words = []
    for i in range(1, len(titles_list)):
        if i != index_high_len:
            words = titles_list[i].split()
            new_words = [word for word in words if word not in highest_len_title_words]
            # new_words = [word for word in new_words if word not in stop_words]
            total_new_words += new_words
    return list(set(total_new_words))


In [ ]:
import pickle

agg_s = list(agg_searches.keys())
with open("./data/torob_list_search", "wb") as fp:
    pickle.dump(agg_s, fp)
# with open ('/content/drive/MyDrive/torob_list_search', 'rb') as fp:
# agg_s = pickle.load(fp)


In [ ]:
conn = sqlite3.connect("my_database.db")
c = conn.cursor()
data_df = pd.DataFrame(columns=["query", "p_id", "p_des", "label"])
for query in tqdm_notebook(agg_s):
    results = agg_searches[query]
    for product_id, res_clicks in results["results"].most_common(50):
        if product_id != None:
            c.execute("SELECT * FROM products WHERE id = ?", (product_id,))
            result_product = c.fetchone()
            titles_list_product = json.loads(result_product[2])
            if len(titles_list_product) > 0:
                highest_len_product = max(titles_list_product, key=len)
                index_highest_len_product = titles_list_product.index(
                    highest_len_product
                )
                product_title_new_words = find_new_words(
                    titles_list_product, index_highest_len_product
                )
                p_des = " ".join(
                    [highest_len_product, " ".join(product_title_new_words)]
                )
                candidate_score = results["clicks"].get(product_id, 0)
                candidate_score = np.log2(candidate_score + 1)
                data_df = data_df.append(
                    {
                        "query": query,
                        "p_id": product_id,
                        "p_des": p_des,
                        "label": candidate_score,
                    },
                    ignore_index=True,
                )


In [ ]:
data_df.to_csv("./data_trob20.csv", sep="\t", encoding="utf-8")


In [ ]:
len(data_df)

In [ ]:
len(data_df)

In [ ]:
data_df.head()

In [ ]:
conn.commit()
conn.close()

In [ ]:
sum(data_df["label"] > 1) / len(data_df["label"])

In [ ]:
len(agg_s)

In [ ]:
agg_searches[agg_s[0]]

In [ ]:
def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.

    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, "r") as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)


In [ ]:
product = pd.DataFrame(read_json_lines("./data/products-info_v1.jsonl"))


In [ ]:
product = product.set_index("id")


In [ ]:
product.loc[1867826]["titles"]


In [ ]:
data_df = pd.DataFrame(columns=["query", "p_id", "p_des", "label"])
for query in tqdm_notebook(agg_s):
    results = agg_searches[query]
    for product_id, res_clicks in results["results"].most_common(50):
        if product_id != None:
            titles_list_product = product.loc[product_id]["titles"]
            if len(titles_list_product) > 0:
                highest_len_product = max(titles_list_product, key=len)
                index_highest_len_product = titles_list_product.index(
                    highest_len_product
                )
                product_title_new_words = find_new_words(
                    titles_list_product, index_highest_len_product
                )
                p_des = " ".join(
                    [highest_len_product, " ".join(product_title_new_words)]
                )
                candidate_score = results["clicks"].get(product_id, 0)
                candidate_score = np.log2(candidate_score + 1)
                data_df = data_df.append(
                    {
                        "query": query,
                        "p_id": product_id,
                        "p_des": p_des,
                        "label": candidate_score,
                    },
                    ignore_index=True,
                )


product.loc[1867826]

In [ ]:
product_info.head()


In [ ]:
data_df.to_csv("./data_trob_without_sql.csv", sep="\t", encoding="utf-8")

In [ ]:
data_df.head()

In [ ]:
def read_json_lines(path, n_lines=None):
    """Creates a generator which reads and returns lines of
    a json lines file, one line at a time, each as a dictionary.

    This could be used as a memory-efficient alternative of `pandas.read_json`
    for reading a json lines file.
    """
    with open(path, "r") as f:
        for i, line in enumerate(f):
            if n_lines == i:
                break
            yield json.loads(line)

In [ ]:
product = pd.DataFrame(read_json_lines("./data/products-info_v1.jsonl"))

In [ ]:
product = product.set_index("id")

In [ ]:
product.loc[1867826].min_price

In [ ]:
agg_s[:10]

In [ ]:
agg_searches[agg_s[0]]


In [ ]:
def find_new_words(titles_list, index_high_len):
    highest_len_title_words = titles_list[index_high_len].split()
    total_new_words = []
    for i in range(1, len(titles_list)):
        if i != index_high_len:
            words = titles_list[i].split()
            new_words = [word for word in words if word not in highest_len_title_words]
            new_words = [word for word in new_words if word not in stopwords]
            total_new_words += new_words
    return list(set(total_new_words))


In [ ]:
data_df = pd.DataFrame(columns=["query", "p_id", "p_des", "label"])
for query in tqdm_notebook(agg_s):

    results = agg_searches[query]
    max_num_shops = []
    for product_id, res_clicks in results["results"].most_common(60):
        if product_id != None:
            max_num_shop = product.loc[product_id].max_num_shops
            if max_num_shops != None:
                max_num_shops.append(max_num_shop)
        mean_max_shops = np.mean(max_num_shops)
    for product_id, res_clicks in results["results"].most_common(60):
        if product_id != None:
            titles_list_product = product.loc[product_id]["titles"]
            category_name = result_product[1]

            if len(titles_list_product) > 0:
                max_shops = product.loc[product_id].max_num_shops
                popularity = (
                    "popular" if (max_shops) > (mean_max_shops + 1) else "unpopular"
                )
                highest_len_product = max(titles_list_product, key=len)
                index_highest_len_product = titles_list_product.index(
                    highest_len_product
                )
                product_title_new_words = find_new_words(
                    titles_list_product, index_highest_len_product
                )

                candidate_score = results["clicks"].get(product_id, 0)
                candidate_score = np.log2(candidate_score + 1)
                clicks = results["clicks"].get(product_id, 0)
                impressions = results["results"].get(product_id, 0)
                ctr = (clicks + 1) / (impressions + len(results["results"]))
                max_clicks = np.max(list(results["clicks"].values()))
                data_df = data_df.append(
                    {
                        "query": query,
                        "category_name": category_name,
                        "p_id": product_id,
                        "new words": product_title_new_words,
                        "p_des": highest_len_product,
                        "label": candidate_score,
                        "clicks": clicks,
                        "impressions": impressions,
                        "ctr": ctr,
                        "mean_max_shops": mean_max_shops,
                        "popularity": popularity,
                        "log_normalized_candidate_score": np.log2(clicks + 1)
                        / np.log2(max_clicks),
                        "min_price": product.loc[product_id].min_price,
                    },
                    ignore_index=True,
                )


In [ ]:
data_df
# temp_df = data_df.copy(deep=True)

In [ ]:
# as csv
data_df.to_csv("data_df.csv", index=False)

In [ ]:
# as Pickle
import pickle
with open('data_df.pkl', 'wb') as f:
    pickle.dump(data_df, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
clicks_j = click_data[queries[i]][products[i][j]]["clicks"]
impressions_j = click_data[queries[i]][products[i][j]]["impressions"]
ctr_j = clicks_j / impressions_j
clicks_k = click_data[queries[i]][products[i][k]]["clicks"]
impressions_k = click_data[queries[i]][products[i][k]]["impressions"]
ctr_k = clicks_k / impressions_k
rel_j = np.ceil(4 * ctr_j / max(ctr_j, ctr_k))
rel_k = np.ceil(4 * ctr_k / max(ctr_j, ctr_k))


In [ ]:
# extract 10 queries with the highest number of products
top_10_queries = data_df.groupby("query").count().sort_values("p_id", ascending=False)[:10].index

# extract the data for the top 10 queries
top_10_queries_data = data_df[data_df["query"].isin(top_10_queries)]
# extract the data for the top 10 queries with the highest number of products
top_10_queries_highest_products = data_df.groupby("query").count().sort_values(
    "p_id", ascending=False
)[:10]
# extract the data for the top 10 queries with the highest number of products
top_10_queries_highest_products_data = data_df[
    data_df["query"].isin(top_10_queries_highest_products.index)
]

In [ ]:
# relevance score is calculated as follows:  rel_{ctr}(q,d) = ceil(4.\frac{ctr(q,d)}{max_{d \in D_{q}} ctr(q,d)}), it's scaled to the range [0,4]
# where ctr(q,d) is the click-through rate of document d for query q, and D_{q} is the set of documents for query q.
# $ctr(q,d) = \frac{clicks(q,d)}{impressions(q,d)}$, $D_{q}$ is the set of products to be ranked for a query $q$. We sample \textbf{2} weeks of click-stream data for fine-tuning our models. We consider only those products in the result set of a query that had received at least \textbf{50} impressions to ensure reliable CTR estimates.

# add relevance score for each product
data_df["relevance"] = 0
# group by query
for query in tqdm_notebook(data_df["query"].unique()):
    # get the products for the query
    products = data_df[data_df["query"] == query]["p_id"].unique()
    # get the clicks and impressions for the products
    clicks = data_df[data_df["query"] == query]["clicks"].values
    impressions = data_df[data_df["query"] == query]["impressions"].values
    # calculate ctr
    ctr = clicks / impressions
    # calculate max ctr
    max_ctr = np.max(ctr)
    # calculate relevance score
    relevance = np.ceil(4 * ctr / max_ctr)
    # add relevance score to data_df
    data_df.loc[data_df["query"] == query, "relevance"] = relevance



In [1]:
%reset -f

In [3]:
import numpy as np
import pandas as pd

In [4]:
temp_df = pd.read_csv("data_df.csv")
temp_df

,query,p_id,p_des,label,category_name,clicks,ctr,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity
0,لوستر سقفی برنز,6462477,لوستر سقفی چشمه نور 5 شعله کد C2542-B برنز,2.0,کاپشن، بارانی و پالتو زنانه,3.0,0.012945,22.0,1.00000,1.440678,821000.0,[],unpopular
1,لوستر سقفی برنز,7385791,لوستر سقفی چشمه نور 3 شعله کد C2542/3B برنز,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003236,22.0,0.00000,1.440678,792000.0,['۳'],unpopular
2,لوستر سقفی برنز,3775536,لوستر سقفی مدرن مدل نیلوفر سه تایی- سفید,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003279,18.0,0.00000,1.440678,400000.0,[],unpopular
3,لوستر سقفی برنز,1663988,لوستر سقفی کریستالی دایره سایز 25 مدل کارمانیا,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003279,18.0,0.00000,1.440678,756000.0,[],unpopular
4,لوستر سقفی برنز,6946932,لوستر سقفی کریستالی دایره سایز 25 دوبلکس مدل آداس,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003279,18.0,0.00000,1.440678,756000.0,[],unpopular
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1571356,کاپشن زنانه ترک,4987530,کاپشن زنانه ترک کد HBIN 80,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.007092,2.0,0.00000,1.000000,1602840.0,[],unpopular
1571357,کاپشن زنانه ترک,5323788,خرید کاپشن زنانه ترک برند Koton رنگ مشکی کد ty...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.007092,2.0,0.00000,1.000000,1000000.0,"['Dik', 'Yaka', 'Mont', 'Şişme', 'Fermuarlı']",unpopular
1571358,کاپشن زنانه ترک,6993184,خرید نقدی کاپشن زنانه ترک برند TAMER COLLECTİ...,1.0,کاپشن، بارانی و پالتو زنانه,1.0,0.014184,2.0,0.63093,1.000000,1920000.0,"['Vizon', 'Mont', 'Kapüşonlu', 'Beden', 'Ultra...",unpopular
1571359,کاپشن زنانه ترک,7938070,Kahverengi Oversize Çıt Çıt Kapamalı Kapitone ...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.007092,2.0,0.00000,1.000000,840000.0,"['ty195352872', 'رنگ', 'قهوه', 'کد', 'ترک', 'پ...",unpopular


In [ ]:
# unique 

In [ ]:
# set relevance score to int and zero
# temp_df["relevance"] = 0

In [8]:
# rename ctr to ctr_score
temp_df.rename(columns={"ctr": "ctr_score"}, inplace=True)

In [9]:
# calculate ctr for each product
temp_df["ctr_score"] = temp_df["clicks"] / temp_df["impressions"]


In [16]:
np.unique(temp_df["ctr"].values, return_counts=True)

(array([4.89715965e-04, 4.98256104e-04, 5.16528926e-04, ...,
        9.05898876e-01, 1.00787402e+00, 1.13392857e+00]),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

array([6462477, 7385791, 3775536, ..., 6993184, 7938070, 8097039],
      dtype=int64)

In [18]:
# how many products are ctr = 0


,query,p_id,p_des,label,category_name,clicks,ctr,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity


In [10]:
ctr_zero

0

In [11]:
# only consider products that have at least 50 impressions
temp_df = temp_df[temp_df["impressions"] >= 50]

In [12]:
temp_df

,query,p_id,p_des,label,category_name,clicks,ctr_score,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,ctr
59,ساعت هوشمند,9391819,ساعت هوشمند مدل T55 WATCH به همراه‌ یک عدد بند...,11.448633,کاپشن، بارانی و پالتو زنانه,2794.0,0.204657,10060.0,1.000045,72.542373,245000.0,"['watch', 'Watch', 'رنگ', 'همراه', '5', 'اپل',...",unpopular,0.277734
60,ساعت هوشمند,8563833,ساعت هوشمند شیائومی هایلو مدل Haylou LS02 Glob...,10.246741,کاپشن، بارانی و پالتو زنانه,1214.0,0.089496,9979.0,0.895059,72.542373,527000.0,"['سولار', 'haylou', 'مدلHaylou', 'اصل', 'کمپان...",popular,0.121655
61,ساعت هوشمند,2459592,ساعت هوشمند شیائومی مدل Mibro Lite XPAW004 ا X...,11.157978,کاپشن، بارانی و پالتو زنانه,2284.0,0.168797,9940.0,0.974656,72.542373,145000.0,"['10', 'watch', 'رنگ', 'همراه', 'شیائومیmibro'...",popular,0.229779
62,ساعت هوشمند,7824893,سیلیکون Mi Smart Band 6 - 1.56 اینچ با صفحه نم...,9.870365,کاپشن، بارانی و پالتو زنانه,935.0,0.069144,9940.0,0.862182,72.542373,426000.0,"['بروزکالا', '44mm', 'هوسمند', '18', 'میبند', ...",popular,0.094064
63,ساعت هوشمند,9901900,ساعت هوشمند شیائومی مدل Mibro X1 – نسخه گلوبال...,9.434628,کاپشن، بارانی و پالتو زنانه,691.0,0.051127,9938.0,0.824121,72.542373,969000.0,"['10', 'بروزکالا', 'ATM', 'watch', 'Watch', 'ض...",popular,0.069531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565859,buds 2 پرو,4723271,هدفون کی زد مدل ZS10 PRO,1.000000,کاپشن، بارانی و پالتو زنانه,1.0,0.012821,84.0,0.147180,15.694915,1700000.0,[],unpopular,0.011905
1565860,buds 2 پرو,6254931,هندزفری بلوتوث دوگوش شیائومی Xiaomi Redmi Buds...,1.000000,کاپشن، بارانی و پالتو زنانه,1.0,0.012903,83.0,0.147180,15.694915,835000.0,"['18', 'Graphite', 'redmi', '(Buds', '(Glacier...",popular,0.012048
1565861,buds 2 پرو,4760866,هندزفری بلوتوثی شیائومی Xiaomi Redmi Buds 4 Pr...,1.584963,کاپشن، بارانی و پالتو زنانه,2.0,0.019868,79.0,0.233274,15.694915,2349000.0,"['پرو', 'مدل', 'ردمی', 'M2132E1', 'xiaomi', 'ه...",unpopular,0.025316
1565862,buds 2 پرو,5228927,هندزفری بلوتوث دوگوش شیائومی Xiaomi 1More Pist...,0.000000,کاپشن، بارانی و پالتو زنانه,0.0,0.006667,78.0,0.000000,15.694915,999000.0,"['Piston', 'PistoneBuds', 'PRO', 'گلوبال', 'Pi...",popular,0.000000


In [20]:
grouped = temp_df.set_index("p_id").groupby("query")


In [21]:
test = temp_df.copy(deep=True)

In [22]:
test.set_index("p_id", inplace=True)

In [37]:
test

,query,p_des,label,category_name,clicks,ctr,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,ctr_score
p_id,,,,,,,,,,,,,
6462477,لوستر سقفی برنز,لوستر سقفی چشمه نور 5 شعله کد C2542-B برنز,2.0,کاپشن، بارانی و پالتو زنانه,3.0,0.012945,22.0,1.00000,1.440678,821000.0,[],unpopular,0.136364
7385791,لوستر سقفی برنز,لوستر سقفی چشمه نور 3 شعله کد C2542/3B برنز,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003236,22.0,0.00000,1.440678,792000.0,['۳'],unpopular,0.000000
3775536,لوستر سقفی برنز,لوستر سقفی مدرن مدل نیلوفر سه تایی- سفید,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003279,18.0,0.00000,1.440678,400000.0,[],unpopular,0.000000
1663988,لوستر سقفی برنز,لوستر سقفی کریستالی دایره سایز 25 مدل کارمانیا,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003279,18.0,0.00000,1.440678,756000.0,[],unpopular,0.000000
6946932,لوستر سقفی برنز,لوستر سقفی کریستالی دایره سایز 25 دوبلکس مدل آداس,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.003279,18.0,0.00000,1.440678,756000.0,[],unpopular,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987530,کاپشن زنانه ترک,کاپشن زنانه ترک کد HBIN 80,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.007092,2.0,0.00000,1.000000,1602840.0,[],unpopular,0.000000
5323788,کاپشن زنانه ترک,خرید کاپشن زنانه ترک برند Koton رنگ مشکی کد ty...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.007092,2.0,0.00000,1.000000,1000000.0,"['Dik', 'Yaka', 'Mont', 'Şişme', 'Fermuarlı']",unpopular,0.000000
6993184,کاپشن زنانه ترک,خرید نقدی کاپشن زنانه ترک برند TAMER COLLECTİ...,1.0,کاپشن، بارانی و پالتو زنانه,1.0,0.014184,2.0,0.63093,1.000000,1920000.0,"['Vizon', 'Mont', 'Kapüşonlu', 'Beden', 'Ultra...",unpopular,0.500000


In [38]:
# max number of clicks in test df
max_clicks = np.max(test["clicks"].values)

In [39]:
max_clicks

9220.0

In [25]:
grouped = test.groupby("query")

In [26]:
grouped.get_group("10 a")

,query,p_des,label,category_name,clicks,ctr,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity
p_id,,,,,,,,,,,,
7224899,10 a,گوشی موبایل شیائومی مدل REDMI 10A دو سیم‌ کار...,6.714246,کاپشن، بارانی و پالتو زنانه,104.0,0.319149,178.0,1.002060,87.6,12000.0,"['(کارتن', '10', 'اصلی)', '3', '4GB/64GB', '10...",unpopular
3243040,10 a,گوشی موبایل شیائومی مدل Redmi 10A دوسیم کارت ب...,5.321928,کاپشن، بارانی و پالتو زنانه,39.0,0.121581,178.0,0.794265,87.6,3039000.0,"['220233L2G', '10', 'رم2', 'حافظه', '–', '(ارس...",unpopular
3410497,10 a,گوشی موبایل شیائومی مدل Xiaomi REDMI 10A 22023...,5.954196,کاپشن، بارانی و پالتو زنانه,61.0,0.188450,178.0,0.888628,87.6,12000.0,"['220233L2G', '10', 'اصلی)', 'و3', '–', '(ارسا...",unpopular
707374,10 a,گوشی موبایل سامسونگ مدل Galaxy A32 4G دو سیم ک...,2.000000,کاپشن، بارانی و پالتو زنانه,3.0,0.012158,178.0,0.298488,87.6,620000.0,"['تکرارری', 'باحافظه128گیگ', 'مدلGalaxy', 'Rom...",popular
6133410,10 a,گوشی موبایل سامسونگ مدل Galaxy A13 دو سیم کارت...,2.000000,کاپشن، بارانی و پالتو زنانه,3.0,0.012158,178.0,0.298488,87.6,434000.0,"['گیگابایت,', '(کارمندی', 'Storage/4GB', 'Blue...",popular
6013462,10 a,گوشی موبایل اپل مدل iPhone 13 Pro Max Not Acti...,0.000000,کاپشن، بارانی و پالتو زنانه,0.0,0.003049,177.0,0.000000,87.6,5430000.0,"['(18', '(نات', 'iphone', '(JA', 'Alpine', 'za...",popular
9563634,10 a,گوشی موبایل شیائومی مدل Redmi 10A دوسیم کارت ب...,3.906891,کاپشن، بارانی و پالتو زنانه,14.0,0.045872,176.0,0.583080,87.6,3970000.0,"['220233L2G', '10', '–', 'حافظه', '(ارسال', '4...",unpopular
6660687,10 a,گوشی موبایل سامسونگ مدل Galaxy A53 5G دو سیم ک...,1.000000,کاپشن، بارانی و پالتو زنانه,1.0,0.006250,169.0,0.149244,87.6,1146249.0,"['گیگابایت,', '256گیگابایت', '18', '8)', 'SM',...",popular
8560034,10 a,گوشی موبایل سامسونگ Galaxy A10S دو سیم کارت با...,1.000000,کاپشن، بارانی و پالتو زنانه,1.0,0.007273,124.0,0.149244,87.6,2000000.0,"['32گیگ', '32gb', 'Rom', '۳۲', '18', '۳۲گیگابا...",unpopular


In [36]:
# max number of clicks in test
max_clicks = np.max(list(test["clicks"].values()))

TypeError: 'numpy.ndarray' object is not callable

In [32]:
test["ctr_score"] = test["clicks"] / test["impressions"]


In [33]:
# how many zero ctr
np.unique(test["ctr_score"].values, return_counts=True)

(array([0.00000000e+00, 1.22100122e-03, 1.51515152e-03, ...,
        4.00000000e+00, 5.00000000e+00, 5.33333333e+00]),
 array([960111,      1,      1, ...,     10,      1,      1], dtype=int64))

In [35]:
# get the queries with the highest number of products
queries_with_highest_number_of_products = grouped.count().sort_values(
    "p_des", ascending=False
)[:10].index

# get the data for the queries with the highest number of products
queries_with_highest_number_of_products_data = test[
    test["query"].isin(queries_with_highest_number_of_products)
]

In [27]:
# calculate max_ctr for each query
max_ctr = grouped.apply(lambda x: np.max(x["ctr"]))

In [28]:
# get min_ctr for each query
min_ctr = grouped.apply(lambda x: np.min(x["ctr"]))

In [31]:
min_ctr

query
0022            0.033333
01              0.017241
0180            0.022727
03              0.015873
070             0.031250
                  ...   
یوکللی          0.007576
یویو حرفه‌ای    0.007463
یویو فلزی       0.011236
یک نفره         0.004082
یکبار مصرف      0.003448
Length: 28624, dtype: float64

In [29]:
# max_ctr

# show the max_ctr sorted by value.
max_ctr.sort_values(ascending=False)

query
dsp 90286                   1.133929
هیرویت                      1.007874
qcy t 13                    0.905899
فلویید مای                  0.885714
a 73 128                    0.883721
                              ...   
ساعت مچی اتوماتیک مردانه    0.004792
لویی ویتون                  0.004666
ادوپرفیوم                   0.004525
نخ و نقشه                   0.004323
مادام کوکو                  0.004032
Length: 28624, dtype: float64

In [30]:
np.unique(max_ctr.sort_values(ascending=False), return_counts=True)

(array([0.00403226, 0.00432277, 0.00452489, ..., 0.90589888, 1.00787402,
        1.13392857]),
 array([1, 1, 1, ..., 1, 1, 1], dtype=int64))

In [42]:
test[test["query"] == "samsung"]

,query,p_des,label,category_name,clicks,ctr_score,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,ctr
p_id,,,,,,,,,,,,,
6660687,samsung,گوشی موبایل سامسونگ مدل Galaxy A53 5G دو سیم ک...,5.321928,کاپشن، بارانی و پالتو زنانه,39.0,0.063492,142.0,1.006911,91.779661,1146249.0,"['گیگابایت,', '256گیگابایت', '18', '8)', 'SM',...",popular,0.274648
983448,samsung,گوشی موبایل سامسونگ مدل Galaxy A52 رنگ: مشکی -...,3.906891,کاپشن، بارانی و پالتو زنانه,14.0,0.023810,142.0,0.739185,91.779661,8150000.0,"['256گیگابایت', '8GB)', 'A52باحافظه256گیگ', 'R...",popular,0.098592
6133410,samsung,گوشی موبایل سامسونگ مدل Galaxy A13 دو سیم کارت...,4.857981,کاپشن، بارانی و پالتو زنانه,28.0,0.046105,141.0,0.919132,91.779661,434000.0,"['گیگابایت,', '(کارمندی', 'Storage/4GB', 'Blue...",popular,0.198582
707374,samsung,گوشی موبایل سامسونگ مدل Galaxy A32 4G دو سیم ک...,4.459432,کاپشن، بارانی و پالتو زنانه,21.0,0.034976,141.0,0.843726,91.779661,620000.0,"['تکرارری', 'باحافظه128گیگ', 'مدلGalaxy', 'Rom...",popular,0.148936
4966652,samsung,گوشی موبایل سامسونگ مدل Galaxy S22 Ultra 5G دو...,4.584963,کاپشن، بارانی و پالتو زنانه,23.0,0.038217,140.0,0.867477,91.779661,4100000.0,"['s22', '18', 'داخلی/12GB', 'موبايل', 'and', '...",popular,0.164286
2334880,samsung,گوشی سامسونگ Galaxy A03s ظرفیت 64 گیگابایت رم ...,4.247928,کاپشن، بارانی و پالتو زنانه,18.0,0.030303,139.0,0.803709,91.779661,2685000.0,"['نیازی', 'Rom', '18', 'Storage/4GB', 'Blue,',...",popular,0.129496
6967573,samsung,گوشی موبایل سامسونگ مدل Galaxy S21 FE 5G دو سی...,3.169925,کاپشن، بارانی و پالتو زنانه,8.0,0.014563,130.0,0.599751,91.779661,1773000.0,"['256گیگابایت', '8)', 'گیگابایت-', 'Storage/8G...",popular,0.061538
814754,samsung,گوشی موبایل سامسونگ مدل Galaxy A03 Core دو س...,3.700440,کاپشن، بارانی و پالتو زنانه,12.0,0.021207,125.0,0.700125,91.779661,274800.0,"['SM-A032', 'SM-A032F/DS', 'مدلGalaxy', 'Samsu...",popular,0.096000
6076957,samsung,گوشی موبایل سامسونگ مدل Galaxy A13 دو سیم کارت...,2.807355,کاپشن، بارانی و پالتو زنانه,6.0,0.011438,124.0,0.531153,91.779661,2000.0,"['گیگابایت,', '18', 'Storage/4GB', 'گیگابایت-'...",popular,0.048387


In [44]:
# show query data for the query "بازی فکری جنگل مخوف"
test[test["query"] == "بازی"].sort_values("ctr", ascending=False)

,query,p_des,label,category_name,clicks,ctr_score,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,ctr
p_id,,,,,,,,,,,,,
4693461,بازی,فرمان مسابقه‌ای Logitech مدل G29 سال تولید ۲۰۲...,7.988685,کاپشن، بارانی و پالتو زنانه,253.0,0.204180,796.0,1.000713,39.847458,7250000.0,"['PlayStation/PC', '10', 'PS5', 'کامپیوتر', 'D...",unpopular,0.317839
5133085,بازی,بازی جی تی ای 5 (GTA 5) فارسی مخصوص کامپیوتر,6.285402,کاپشن، بارانی و پالتو زنانه,77.0,0.101430,321.0,0.787349,39.847458,116000.0,"['GTA', 'خرید', 'زیرنویس', 'نسخه', 'V', 'gta',...",unpopular,0.239875
8768238,بازی,کنسول بازی دستی مدل SUP 400 in 1 - ضمانت تعویض...,6.832890,کاپشن، بارانی و پالتو زنانه,113.0,0.117163,525.0,0.855931,39.847458,197000.0,"['blue', 'Sup', 'خرید', 'black', 'Games', 'Bla...",unpopular,0.215238
8663035,بازی,کاکتوس سخنگو شارژی ،کاکتوس رقاص شارژی(((ارسال ...,6.108524,کاپشن، بارانی و پالتو زنانه,68.0,0.088575,331.0,0.765192,39.847458,143000.0,"['عروسک)', 'سفارشی', 'خارجی', 'تودی', 'کن.', '...",popular,0.205438
8630453,بازی,کنسول سونی مدل PlayStation Classic - ضمانت تعو...,7.219169,کاپشن، بارانی و پالتو زنانه,148.0,0.119775,796.0,0.904318,39.847458,689000.0,"['کلاسیک', 'همراه', '5', '1', 'PS1', 'باندل', ...",unpopular,0.185930
7268401,بازی,خرید بازی جی تی ای وی (GTA V) مناسب پلی استیشن 5,5.672425,کاپشن، بارانی و پالتو زنانه,50.0,0.064475,343.0,0.710564,39.847458,998000.0,"['آکبند', 'GTA', 'مخصوص', 'PS5', '–', '(GTA)',...",unpopular,0.145773
4804755,بازی,گیره قلیان دابی برای پلی استیشن 5- GAME AND SM...,6.459432,کاپشن، بارانی و پالتو زنانه,87.0,0.080734,642.0,0.809149,39.847458,170000.0,"['پایه', 'مخصوص', 'خرید', 'مدل', '0573', 'TP5'...",unpopular,0.135514
3488817,بازی,کنسول بازی نینتندو سوییچ لایت رنگ فیروزه ای کن...,5.857981,کاپشن، بارانی و پالتو زنانه,57.0,0.065834,433.0,0.733808,39.847458,5800000.0,"['10', '–', 'Zacian', ')', 'حمل', 'and', 'Gami...",unpopular,0.131640
1444906,بازی,دسته بازی سیم دار تک شوک حرفه ای PANATECH پانا...,4.169925,کاپشن، بارانی و پالتو زنانه,17.0,0.029703,158.0,0.522351,39.847458,167500.0,"['کامپیوتر', 'گيم', 'p-g502p', 'Wired', 'GAME'...",unpopular,0.107595


In [45]:
# grouped.groups
# grouped.apply(lambda x: x.columns)
# grouped.apply(lambda x: x.name)
# for first group
# grouped.apply(lambda x: x.iloc[0])
# grouped.apply(lambda x: x.iloc[0]["ctr"])

# only show first query products
for query in grouped.groups.keys():
    print(query)
    # get products for the query
    products = grouped.get_group(query).reset_index(drop=True)
    # set 
    display(grouped.get_group(query).sort_values("ctr", ascending=False))
    break

10 a


,query,p_des,label,category_name,clicks,ctr_score,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,ctr
p_id,,,,,,,,,,,,,
7224899,10 a,گوشی موبایل شیائومی مدل REDMI 10A دو سیم‌ کار...,6.714246,کاپشن، بارانی و پالتو زنانه,104.0,0.319149,178.0,1.002060,87.6,12000.0,"['(کارتن', '10', 'اصلی)', '3', '4GB/64GB', '10...",unpopular,0.584270
3410497,10 a,گوشی موبایل شیائومی مدل Xiaomi REDMI 10A 22023...,5.954196,کاپشن، بارانی و پالتو زنانه,61.0,0.188450,178.0,0.888628,87.6,12000.0,"['220233L2G', '10', 'اصلی)', 'و3', '–', '(ارسا...",unpopular,0.342697
3243040,10 a,گوشی موبایل شیائومی مدل Redmi 10A دوسیم کارت ب...,5.321928,کاپشن، بارانی و پالتو زنانه,39.0,0.121581,178.0,0.794265,87.6,3039000.0,"['220233L2G', '10', 'رم2', 'حافظه', '–', '(ارس...",unpopular,0.219101
9563634,10 a,گوشی موبایل شیائومی مدل Redmi 10A دوسیم کارت ب...,3.906891,کاپشن، بارانی و پالتو زنانه,14.0,0.045872,176.0,0.583080,87.6,3970000.0,"['220233L2G', '10', '–', 'حافظه', '(ارسال', '4...",unpopular,0.079545
707374,10 a,گوشی موبایل سامسونگ مدل Galaxy A32 4G دو سیم ک...,2.000000,کاپشن، بارانی و پالتو زنانه,3.0,0.012158,178.0,0.298488,87.6,620000.0,"['تکرارری', 'باحافظه128گیگ', 'مدلGalaxy', 'Rom...",popular,0.016854
6133410,10 a,گوشی موبایل سامسونگ مدل Galaxy A13 دو سیم کارت...,2.000000,کاپشن، بارانی و پالتو زنانه,3.0,0.012158,178.0,0.298488,87.6,434000.0,"['گیگابایت,', '(کارمندی', 'Storage/4GB', 'Blue...",popular,0.016854
8560034,10 a,گوشی موبایل سامسونگ Galaxy A10S دو سیم کارت با...,1.000000,کاپشن، بارانی و پالتو زنانه,1.0,0.007273,124.0,0.149244,87.6,2000000.0,"['32گیگ', '32gb', 'Rom', '۳۲', '18', '۳۲گیگابا...",unpopular,0.008065
6660687,10 a,گوشی موبایل سامسونگ مدل Galaxy A53 5G دو سیم ک...,1.000000,کاپشن، بارانی و پالتو زنانه,1.0,0.006250,169.0,0.149244,87.6,1146249.0,"['گیگابایت,', '256گیگابایت', '18', '8)', 'SM',...",popular,0.005917
6013462,10 a,گوشی موبایل اپل مدل iPhone 13 Pro Max Not Acti...,0.000000,کاپشن، بارانی و پالتو زنانه,0.0,0.003049,177.0,0.000000,87.6,5430000.0,"['(18', '(نات', 'iphone', '(JA', 'Alpine', 'za...",popular,0.000000


In [46]:
# get group index
grouped.groups

{'10 a': [7224899, 3243040, 3410497, 707374, 6133410, 6013462, 9563634, 6660687, 8560034, 6076957, 2107002, 2650689], '10 c': [3583373, 3017907, 2345798, 4748879, 5602259, 2650689, 3015638, 6571922, 5147618, 6652460, 7224899, 2387610, 3954813], '11 lite': [4116972, 8642691, 2249796, 3613201, 2367608, 808854, 3542354, 8834697, 7904923, 3211023, 7437752, 9495655, 7283211], '11 lite 5g': [4116972, 8642691, 2249796, 3613201, 808854, 7437752, 3211023], '11 pro max': [5007735, 7862039, 6013462, 4823781, 5582484, 2858342], '11 t': [3211023, 2282190, 1949944, 7437752, 1898330, 799589, 3542354, 743379, 9495655, 7824983, 9501479, 8555218, 4116972, 7587236, 4649342, 2940823, 601864, 3874537, 8642691, 2147296, 2160159, 1889403, 256902, 951730, 933021, 349697, 9223789, 8014723, 8678764, 4357056, 1574650, 3761065, 2114530, 9997703], '11 t 5g': [1898330, 3211023, 7437752, 2282190, 1949944, 799589, 743379, 7824983, 4116972], '11 t پرو 5g': [7437752, 3211023, 1949944, 1898330, 7824983, 799589, 743379, 

In [13]:
# calculate relevance score for each product rel_{ctr}(q,d) = ceil(4.\frac{ctr(q,d)}{max_{d \in D_{q}} ctr(q,d)})
# temp_df["relevance"] = grouped.apply(lambda x: np.ceil(4 * x["ctr"] / np.max(x["ctr"])))
# temp_df["relevance"] = grouped.apply(lambda x: np.ceil(4 * x["ctr"] / max_ctr[x.name]))

In [47]:
relevance_scores = grouped.apply(lambda x: np.ceil(4 * x["ctr"] / np.max(x["ctr"])))

In [48]:
print(relevance_scores[:10])

query  p_id   
10 a   7224899    4.0
       3243040    2.0
       3410497    3.0
       707374     1.0
       6133410    1.0
       6013462    0.0
       9563634    1.0
       6660687    1.0
       8560034    1.0
       6076957    0.0
Name: ctr, dtype: float64


In [49]:
# show unique relevance_scores.values
np.unique(relevance_scores.values, return_counts=True)

(array([ 0.,  1.,  2.,  3.,  4., nan]),
 array([ 7262, 67752, 34174, 11619, 10855,     6], dtype=int64))

In [50]:
# get queries with relevance score "nan"
np.unique(relevance_scores[relevance_scores.isna()].index)

array([('الفا اسلیم', 3680077), ('ساعت نقره زنانه', 7502006),
       ('شیشه جلو 405', 1661566), ('شیشه جلو 405', 7524700),
       ('موتور ساعت دیواری', 4978917), ('موتور ساعت دیواری', 9047486)],
      dtype=object)

In [54]:
relevance_nan = relevance_scores[relevance_scores.isna()].index
for query in relevance_nan:
    print(query[0], query[1])

الفا اسلیم 3680077
ساعت نقره زنانه 7502006
شیشه جلو 405 7524700
شیشه جلو 405 1661566
موتور ساعت دیواری 9047486
موتور ساعت دیواری 4978917


In [56]:
# get relevance_y values from merged_df
relevance_y = relevance_scores[relevance_scores.isna()].values
# relevance_y = relevance_scores[relevance_scores.notna()].values

In [57]:
np.unique(relevance_y, return_counts=True)

(array([nan]), array([6], dtype=int64))

In [63]:
# where index equals to 3680077
test[test["query"] == "الفا اسلیم"]

,query,p_des,label,category_name,clicks,ctr_score,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,ctr
p_id,,,,,,,,,,,,,
3680077,الفا اسلیم,کاور لپ تاپ WiWU مدل Alpha Slim مناسب برای لپ ...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.013514,51.0,0.0,20.363636,750000.0,"['slim', 'کیف', 'اسلیم', 'آلفا', '14', 'wiwu',...",unpopular,0.0


In [64]:
relevance_scores.index

MultiIndex([(    '10 a', 7224899),
            (    '10 a', 3243040),
            (    '10 a', 3410497),
            (    '10 a',  707374),
            (    '10 a', 6133410),
            (    '10 a', 6013462),
            (    '10 a', 9563634),
            (    '10 a', 6660687),
            (    '10 a', 8560034),
            (    '10 a', 6076957),
            ...
            ('یقه اسکی', 3326161),
            ('یقه اسکی', 8753282),
            ( 'یونولیت', 6203654),
            ( 'یونولیت', 3468815),
            ('یونیکورن', 9486172),
            ('یونیکورن', 4360475),
            ('یونیکورن', 4311994),
            ('یونیکورن', 8758736),
            ('یونیکورن',  905923),
            ('یونیکورن', 2514432)],
           names=['query', 'p_id'], length=131668)

In [65]:
relevance_df = pd.DataFrame(index=relevance_scores.index, columns=["relevance"], data=relevance_scores.values)

In [66]:
relevance_df

relevance
query    p_id              
10 a     7224899        4.0
         3243040        2.0
         3410497        3.0
         707374         1.0
         6133410        1.0
...                     ...
یونیکورن 4360475        2.0
         4311994        1.0
         8758736        4.0
         905923         1.0
         2514432        3.0

[131668 rows x 1 columns]

In [67]:
relevance_df.to_csv("relevance_df.csv")

In [2]:
relevance_df = pd.read_csv("relevance_df.csv")

NameError: name 'pd' is not defined

In [69]:
relevance_df.reset_index(inplace=True)

In [71]:
relevance_df.drop(columns=["index"], inplace=True)

In [73]:
relevance_df.set_index("p_id", inplace=True)

In [74]:
relevance_df

,query,relevance
p_id,,
7224899,10 a,4.0
3243040,10 a,2.0
3410497,10 a,3.0
707374,10 a,1.0
6133410,10 a,1.0
...,...,...
4360475,یونیکورن,2.0
4311994,یونیکورن,1.0
8758736,یونیکورن,4.0


In [75]:
print(relevance_df["relevance"].dtype)
print(np.unique(relevance_df["relevance"].values, return_counts=True))

float64
(array([ 0.,  1.,  2.,  3.,  4., nan]), array([ 7262, 67752, 34174, 11619, 10855,     6], dtype=int64))


In [76]:
# replace nan values with 0
relevance_df["relevance"] = relevance_df["relevance"].fillna(0)

In [80]:
relevance_df["relevance"] = relevance_df["relevance"].astype("int64")

In [81]:
relevance_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131668 entries, 7224899 to 2514432
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   query      131668 non-null  object
 1   relevance  131668 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.0+ MB


In [111]:
t_df = test.copy()

# t_df = t_df.set_index(["query", "p_id"])
# relevance_df.reset_index(inplace=True)

In [112]:
t_df["relevance"] = 0

In [113]:
relevance_df

,query,relevance
p_id,,
7224899,10 a,4
3243040,10 a,2
3410497,10 a,3
707374,10 a,1
6133410,10 a,1
...,...,...
4360475,یونیکورن,2
4311994,یونیکورن,1
8758736,یونیکورن,4


In [115]:
len(t_df)

131668

In [116]:
t_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131668 entries, 9391819 to 6302319
Data columns (total 14 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   query                           131668 non-null  object 
 1   p_des                           131668 non-null  object 
 2   label                           131668 non-null  float64
 3   category_name                   131668 non-null  object 
 4   clicks                          131668 non-null  float64
 5   ctr_score                       131668 non-null  float64
 6   impressions                     131668 non-null  float64
 7   log_normalized_candidate_score  131668 non-null  float64
 8   mean_max_shops                  131668 non-null  float64
 9   min_price                       130651 non-null  float64
 10  new words                       131668 non-null  object 
 11  popularity                      131668 non-null  object 
 12  ctr      

In [118]:
relevance_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 131668 entries, 7224899 to 2514432
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   query      131668 non-null  object
 1   relevance  131668 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 7.0+ MB


In [119]:
relevance_df

,query,relevance
p_id,,
7224899,10 a,4
3243040,10 a,2
3410497,10 a,3
707374,10 a,1
6133410,10 a,1
...,...,...
4360475,یونیکورن,2
4311994,یونیکورن,1
8758736,یونیکورن,4


In [1]:
# add relevance score to t_df
t_df.loc[relevance_df.index, "relevance"] = relevance_df["relevance"]

NameError: name 'relevance_df' is not defined

In [ ]:
t_df

,query,p_des,label,category_name,clicks,ctr_score,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,ctr,relevance
p_id,,,,,,,,,,,,,,
9391819,ساعت هوشمند,ساعت هوشمند مدل T55 WATCH به همراه‌ یک عدد بند...,11.448633,کاپشن، بارانی و پالتو زنانه,2794.0,0.204657,10060.0,1.000045,72.542373,245000.0,"['watch', 'Watch', 'رنگ', 'همراه', '5', 'اپل',...",unpopular,0.277734,0
8563833,ساعت هوشمند,ساعت هوشمند شیائومی هایلو مدل Haylou LS02 Glob...,10.246741,کاپشن، بارانی و پالتو زنانه,1214.0,0.089496,9979.0,0.895059,72.542373,527000.0,"['سولار', 'haylou', 'مدلHaylou', 'اصل', 'کمپان...",popular,0.121655,0
2459592,ساعت هوشمند,ساعت هوشمند شیائومی مدل Mibro Lite XPAW004 ا X...,11.157978,کاپشن، بارانی و پالتو زنانه,2284.0,0.168797,9940.0,0.974656,72.542373,145000.0,"['10', 'watch', 'رنگ', 'همراه', 'شیائومیmibro'...",popular,0.229779,0
7824893,ساعت هوشمند,سیلیکون Mi Smart Band 6 - 1.56 اینچ با صفحه نم...,9.870365,کاپشن، بارانی و پالتو زنانه,935.0,0.069144,9940.0,0.862182,72.542373,426000.0,"['بروزکالا', '44mm', 'هوسمند', '18', 'میبند', ...",popular,0.094064,0
9901900,ساعت هوشمند,ساعت هوشمند شیائومی مدل Mibro X1 – نسخه گلوبال...,9.434628,کاپشن، بارانی و پالتو زنانه,691.0,0.051127,9938.0,0.824121,72.542373,969000.0,"['10', 'بروزکالا', 'ATM', 'watch', 'Watch', 'ض...",popular,0.069531,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4723271,buds 2 پرو,هدفون کی زد مدل ZS10 PRO,1.000000,کاپشن، بارانی و پالتو زنانه,1.0,0.012821,84.0,0.147180,15.694915,1700000.0,[],unpopular,0.011905,0
6254931,buds 2 پرو,هندزفری بلوتوث دوگوش شیائومی Xiaomi Redmi Buds...,1.000000,کاپشن، بارانی و پالتو زنانه,1.0,0.012903,83.0,0.147180,15.694915,835000.0,"['18', 'Graphite', 'redmi', '(Buds', '(Glacier...",popular,0.012048,0
4760866,buds 2 پرو,هندزفری بلوتوثی شیائومی Xiaomi Redmi Buds 4 Pr...,1.584963,کاپشن، بارانی و پالتو زنانه,2.0,0.019868,79.0,0.233274,15.694915,2349000.0,"['پرو', 'مدل', 'ردمی', 'M2132E1', 'xiaomi', 'ه...",unpopular,0.025316,0


In [ ]:
np.unique(t_df["relevance"], return_counts=True)

(array([0, 1, 2, 3, 4], dtype=int64),
 array([ 42499, 476639, 157372,  56885,  85199], dtype=int64))

In [ ]:
relevance_df

,query,relevance
p_id,,
7224899,10 a,4
3243040,10 a,2
3410497,10 a,3
707374,10 a,1
6133410,10 a,1
...,...,...
4360475,یونیکورن,2
4311994,یونیکورن,1
8758736,یونیکورن,4


In [ ]:
# where query_x equals to "10 a" and index is 7224899
t_df[(t_df["query_x"] == "10 a") & (t_df.index == 7224899)]

,query_x,p_des,label,category_name,clicks,ctr_score,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,ctr,query_y,relevance
p_id,,,,,,,,,,,,,,,
7224899,10 a,گوشی موبایل شیائومی مدل REDMI 10A دو سیم‌ کار...,6.714246,کاپشن، بارانی و پالتو زنانه,104.0,0.319149,178.0,1.00206,87.6,12000.0,"['(کارتن', '10', 'اصلی)', '3', '4GB/64GB', '10...",unpopular,0.58427,10 a,4
7224899,10 a,گوشی موبایل شیائومی مدل REDMI 10A دو سیم‌ کار...,6.714246,کاپشن، بارانی و پالتو زنانه,104.0,0.319149,178.0,1.00206,87.6,12000.0,"['(کارتن', '10', 'اصلی)', '3', '4GB/64GB', '10...",unpopular,0.58427,10 c,1
7224899,10 a,گوشی موبایل شیائومی مدل REDMI 10A دو سیم‌ کار...,6.714246,کاپشن، بارانی و پالتو زنانه,104.0,0.319149,178.0,1.00206,87.6,12000.0,"['(کارتن', '10', 'اصلی)', '3', '4GB/64GB', '10...",unpopular,0.58427,a 10,1
7224899,10 a,گوشی موبایل شیائومی مدل REDMI 10A دو سیم‌ کار...,6.714246,کاپشن، بارانی و پالتو زنانه,104.0,0.319149,178.0,1.00206,87.6,12000.0,"['(کارتن', '10', 'اصلی)', '3', '4GB/64GB', '10...",unpopular,0.58427,redmi 10,1
7224899,10 a,گوشی موبایل شیائومی مدل REDMI 10A دو سیم‌ کار...,6.714246,کاپشن، بارانی و پالتو زنانه,104.0,0.319149,178.0,1.00206,87.6,12000.0,"['(کارتن', '10', 'اصلی)', '3', '4GB/64GB', '10...",unpopular,0.58427,redmi 10 a,4
7224899,10 a,گوشی موبایل شیائومی مدل REDMI 10A دو سیم‌ کار...,6.714246,کاپشن، بارانی و پالتو زنانه,104.0,0.319149,178.0,1.00206,87.6,12000.0,"['(کارتن', '10', 'اصلی)', '3', '4GB/64GB', '10...",unpopular,0.58427,redmi 10 c,1
7224899,10 a,گوشی موبایل شیائومی مدل REDMI 10A دو سیم‌ کار...,6.714246,کاپشن، بارانی و پالتو زنانه,104.0,0.319149,178.0,1.00206,87.6,12000.0,"['(کارتن', '10', 'اصلی)', '3', '4GB/64GB', '10...",unpopular,0.58427,ردمی 10,1
7224899,10 a,گوشی موبایل شیائومی مدل REDMI 10A دو سیم‌ کار...,6.714246,کاپشن، بارانی و پالتو زنانه,104.0,0.319149,178.0,1.00206,87.6,12000.0,"['(کارتن', '10', 'اصلی)', '3', '4GB/64GB', '10...",unpopular,0.58427,ردمی 10 a,4
7224899,10 a,گوشی موبایل شیائومی مدل REDMI 10A دو سیم‌ کار...,6.714246,کاپشن، بارانی و پالتو زنانه,104.0,0.319149,178.0,1.00206,87.6,12000.0,"['(کارتن', '10', 'اصلی)', '3', '4GB/64GB', '10...",unpopular,0.58427,گوشی a 10,1


In [ ]:
# get score for index 1074166 in t_df
t_df.iloc[1]

query_x                                                               عطر جیبی
p_des                             عطر مدادی جیبی جامد صابونی وارداتی و باکیفیت
label                                                                 4.523562
category_name                                      کاپشن، بارانی و پالتو زنانه
clicks                                                                    22.0
ctr_score                                                             0.029909
impressions                                                              121.0
log_normalized_candidate_score                                             1.0
mean_max_shops                                                       16.389831
min_price                                                              34000.0
new words                                                            ['ushas']
popularity                                                           unpopular
ctr                                                 

In [ ]:
np.unique(t_df["relevance"].values, return_counts=True)

(array([0, 1, 2, 3, 4], dtype=int64),
 array([444347,   7845,   4329,   1563,   1593], dtype=int64))

In [ ]:
# most relevant products return 100
t_df[t_df["relevance"] == 4].sort_values("ctr", ascending=False)[:100]

,query,p_id,p_des,label,category_name,clicks,ctr,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,relevance
275758,qt 82,148722,هندزفری بلوتوث دوگوش لنوو Lenovo QT82 (((ارسال...,4.392317,کاپشن، بارانی و پالتو زنانه,20.0,1.250000,16.0,1.016287,16.538462,238000.0,"['10', 'Bt', 'QT', 'رنگ', 'Earbuds', '2021', '...",popular,4
452042,galaxy m 52,1276659,گوشی موبایل سامسونگ مدل GALAXY M52 5G SM-M526B...,4.321928,کاپشن، بارانی و پالتو زنانه,19.0,0.950000,20.0,1.017420,34.764706,5249000.0,"['مدلBLACK', '8)', '18', 'SM', '128گیگابایت-',...",popular,4
19677,m 52,1276659,گوشی موبایل سامسونگ مدل GALAXY M52 5G SM-M526B...,10.051209,کاپشن، بارانی و پالتو زنانه,1060.0,0.923345,1148.0,1.000135,20.576271,5249000.0,"['مدلBLACK', '8)', '18', 'SM', '128گیگابایت-',...",popular,4
357203,شیشه فلاسک,361112,انواع شیشه فلاسک از 75 تا 200 هزار تومان,4.392317,کاپشن، بارانی و پالتو زنانه,20.0,0.869565,23.0,1.016287,2.728814,90000.0,[],unpopular,4
278085,2740,641840,دریل بتن کن رونیکس 40 میلی متری - 1250 وات - 7...,4.754888,کاپشن، بارانی و پالتو زنانه,26.0,0.812500,32.0,1.011584,17.000000,5178450.0,"['3', '۲۷۴۰', '–', 'کد', '5', 'شیار۱۲۴۰wپرقدرت...",popular,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7520,گوشی اپل,1013548,گوشی اپل (استوک) iPhone 6s | حافظه 64 گیگابایت...,8.682995,کاپشن، بارانی و پالتو زنانه,410.0,0.282954,1449.0,1.000405,27.152542,2900000.0,"['آیفون', 'iphone', 'گوشی\u200c', 'Smartphone'...",unpopular,4
338757,اتو فلیپس,91519,اتو بخار حرفه ای فیلیپس مدل GC4936 (غیر اصل هل...,3.000000,کاپشن، بارانی و پالتو زنانه,7.0,0.280000,25.0,1.068622,18.406780,2100000.0,"['(PHILIPS)', 'دستی', 'هلندمونتاژ', '3000وات',...",popular,4
276767,سبد پلاستیکی,665876,سبد پلاستیکی RING رنگ سفید ۵۲۵(ساخت ‌ژاپن),3.000000,کاپشن، بارانی و پالتو زنانه,7.0,0.280000,25.0,1.068622,1.627119,39000.0,[],unpopular,4
386006,اهمتر,451363,اهمتر-مولتی متر یاکسون MULTIMETER YAXUN YX-890D,2.584963,کاپشن، بارانی و پالتو زنانه,5.0,0.277778,18.0,0.661642,1.016949,590000.0,[],unpopular,4


In [ ]:
import joblib

In [ ]:
# save with joblib
joblib.dump(t_df, "train_data_df.pkl")

['train_data_df.pkl']

In [ ]:
t_df.to_csv("train_data_df.csv")

In [ ]:
merged_df = t_df.merge(relevance_df, on=['query', 'p_id'], how='left')

In [ ]:
print(merged_df.isnull().sum())


query                                  0
p_id                                   0
p_des                                  0
label                                  0
category_name                          0
clicks                                 0
ctr                                    0
impressions                            0
log_normalized_candidate_score         0
mean_max_shops                         0
min_price                           7599
new words                              0
popularity                             0
relevance_x                       459677
relevance_y                       459677
dtype: int64


In [ ]:
t_df

,query,p_id,p_des,label,category_name,clicks,ctr,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,relevance
0,لوستر سقفی برنز,6462477,لوستر سقفی چشمه نور 5 شعله کد C2542-B برنز,2.0,کاپشن، بارانی و پالتو زنانه,3.0,0.136364,22.0,1.0,1.440678,821000.0,[],unpopular,0
1,لوستر سقفی برنز,7385791,لوستر سقفی چشمه نور 3 شعله کد C2542/3B برنز,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,22.0,0.0,1.440678,792000.0,['۳'],unpopular,0
2,لوستر سقفی برنز,3775536,لوستر سقفی مدرن مدل نیلوفر سه تایی- سفید,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,18.0,0.0,1.440678,400000.0,[],unpopular,0
3,لوستر سقفی برنز,1663988,لوستر سقفی کریستالی دایره سایز 25 مدل کارمانیا,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,18.0,0.0,1.440678,756000.0,[],unpopular,0
4,لوستر سقفی برنز,6946932,لوستر سقفی کریستالی دایره سایز 25 دوبلکس مدل آداس,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,18.0,0.0,1.440678,756000.0,[],unpopular,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459672,همزن گوسونیک 407,8061315,همزن گوسونیک مدل GSM-889 ا Gosonic GSM-889 Sta...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,29.0,0.0,21.000000,2580000.0,"['گاسونیک', 'W', 'gsm889', 'B', '۸۸۹', 'برقی',...",unpopular,0
459673,همزن گوسونیک 407,264504,همزن دستی گیربوکسی گاسونیک مدل 839 با پایه نگه...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,29.0,0.0,21.000000,510000.0,"['Gosonic', 'Mixer', 'برقی', 'برند', 'hand', '...",popular,0
459674,همزن گوسونیک 407,4273225,همزن کاسه دار حرفه ای برند گوسونیک مدل Gosonic...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,29.0,0.0,21.000000,1040000.0,"['لیتر', '–', 'کاسه\u200cدار', 'کد', 'گاسونیک'...",popular,0
459675,همزن گوسونیک 407,6439966,همزن برقی حرفه ای کاسه دار گوسونیک مدل Gosonic...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,29.0,0.0,21.000000,3199000.0,"['8.5', 'electric', 'Mixer', 'GSM908', 'Stand'...",unpopular,0


In [ ]:
t_df = t_df.merge(relevance_df, on=["query", "p_id"], how="left")

In [ ]:
t_df

,query,p_id,p_des,label,category_name,clicks,ctr,impressions,log_normalized_candidate_score,mean_max_shops,min_price,new words,popularity,level_0,index,relevance
0,لوستر سقفی برنز,6462477,لوستر سقفی چشمه نور 5 شعله کد C2542-B برنز,2.0,کاپشن، بارانی و پالتو زنانه,3.0,0.136364,22.0,1.0,1.440678,821000.0,[],unpopular,NaN,NaN,NaN
1,لوستر سقفی برنز,7385791,لوستر سقفی چشمه نور 3 شعله کد C2542/3B برنز,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,22.0,0.0,1.440678,792000.0,['۳'],unpopular,NaN,NaN,NaN
2,لوستر سقفی برنز,3775536,لوستر سقفی مدرن مدل نیلوفر سه تایی- سفید,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,18.0,0.0,1.440678,400000.0,[],unpopular,NaN,NaN,NaN
3,لوستر سقفی برنز,1663988,لوستر سقفی کریستالی دایره سایز 25 مدل کارمانیا,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,18.0,0.0,1.440678,756000.0,[],unpopular,NaN,NaN,NaN
4,لوستر سقفی برنز,6946932,لوستر سقفی کریستالی دایره سایز 25 دوبلکس مدل آداس,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,18.0,0.0,1.440678,756000.0,[],unpopular,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459672,همزن گوسونیک 407,8061315,همزن گوسونیک مدل GSM-889 ا Gosonic GSM-889 Sta...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,29.0,0.0,21.000000,2580000.0,"['گاسونیک', 'W', 'gsm889', 'B', '۸۸۹', 'برقی',...",unpopular,NaN,NaN,NaN
459673,همزن گوسونیک 407,264504,همزن دستی گیربوکسی گاسونیک مدل 839 با پایه نگه...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,29.0,0.0,21.000000,510000.0,"['Gosonic', 'Mixer', 'برقی', 'برند', 'hand', '...",popular,NaN,NaN,NaN
459674,همزن گوسونیک 407,4273225,همزن کاسه دار حرفه ای برند گوسونیک مدل Gosonic...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,29.0,0.0,21.000000,1040000.0,"['لیتر', '–', 'کاسه\u200cدار', 'کد', 'گاسونیک'...",popular,NaN,NaN,NaN
459675,همزن گوسونیک 407,6439966,همزن برقی حرفه ای کاسه دار گوسونیک مدل Gosonic...,0.0,کاپشن، بارانی و پالتو زنانه,0.0,0.000000,29.0,0.0,21.000000,3199000.0,"['8.5', 'electric', 'Mixer', 'GSM908', 'Stand'...",unpopular,NaN,NaN,NaN


In [ ]:
# show 10 products with the highest relevance score
np.unique(merged_df["relevance"].values, return_counts=True)

KeyError: 'relevance'

In [ ]:
# remove the temporary ctr column
# temp_df.drop("ctr", axis=1, inplace=True)

In [ ]:
def handle_zero_division(func):
    """Handle zero division error and return 0 instead of nan."""
    def wrapper(*args, **kwargs):
        try:
            result = func(*args, **kwargs)
        except ZeroDivisionError:
            result = '0'
        return result
    return wrapper

def handle_type_error(func):
    """Handle type error and return 0 instead of nan."""
    def wrapper(*args, **kwargs):
        try:
            result = func(*args, **kwargs)
        except TypeError:
            result = '0'
        return result
    return wrapper

@handle_zero_division
@handle_type_error
def calculate_relevance(x):
    return np.ceil(4 * x["ctr"] / np.max(x["ctr"]))

test_df = pd.DataFrame({'group': ['A', 'A', 'B', 'B'], 'ctr': [1, 2, 3, 0]})
grouped = test_df.groupby('group')
print(grouped.get_group('B'))

In [ ]:
print(calculate_relevance(grouped.get_group('B')))

In [ ]:
test_df["relevance"] = 0

In [ ]:
try:
    test["relevance"] = grouped.apply(lambda x: np.ceil(4 * x["ctr"] / np.max(x["ctr"])))
except TypeError as e:
    if str(e) == "incompatible index of inserted column with frame index":
        print("Caught the 'incompatible index of inserted column with frame index' error")
    else:
        raise e

In [ ]:
try:
    test_df["relevance"] = grouped.apply(lambda x: np.ceil(4 * x["ctr"] / np.max(x["ctr"])))
except (ZeroDivisionError, TypeError) as e:
    # Handle the exception here
    print("An error occurred:", e)
